# Strands Agents Demonstration

[Strands Agents](https://github.com/strands-agents/sdk-python) is a simple yet powerful SDK that takes a model-driven approach to building and running AI agents. From simple conversational assistants to complex autonomous workflows, from local development to production deployment, Strands Agents scales with your needs.

Author: Gary A. Stafford

Date: 2025-05-26

References:
- [Stands Agents Blog Post: 'Introducing Strands Agents, an Open Source AI Agents SDK'](https://aws.amazon.com/blogs/opensource/introducing-strands-agents-an-open-source-ai-agents-sdk/)
- [Stands Agents Documentation](https://strandsagents.com/0.1.x/)
- [Stands Agents GitHub Repository](https://github.com/strands-agents/sdk-python)



## Install Strands Agents

In [ ]:
%pip install strands-agents strands-agents-tools boto3 botocore -Uqqq

In [ ]:
# Restart kernel (only works on Linux)
import os

os._exit(00)

In [2]:
# Built-in libraries
import http.client
import json
import logging
import os
import urllib.request

# Third-party libraries
import boto3
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import calculator

# Configure logging
# Set up logging to output to console
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

## Set Your AWS Credentials

There are multiple ways to set your AWS Credentials depending on your environment.

In [ ]:
session = boto3.Session(
    region_name="us-east-1",
    aws_access_key_id="<YOUR_ACCESS_KEY_ID>",
    aws_secret_access_key="<YOUR_SECRET_ACCESS_KEY>",
    aws_session_token="<YOUR_SESSION_TOKEN>",
)

## Simple Calculator Tool Demonstration

From the Strands Agents GitHub [Quick Start](https://github.com/strands-agents/sdk-python?tab=readme-ov-file#quick-start) documentation.

In [ ]:
%%time

bedrock_model = BedrockModel(
    boto_session=session,
    model_config={
        "model_id": "us.amazon.nova-micro-v1:0",
        "max_tokens": 128,
        "temperature": 0.1,
    },
)

agent = Agent(model=bedrock_model, tools=[calculator])
result = agent("What is the square root of 1764?")
print("\n")

## Custom Tool Demonstration

From the Strands Agents GitHub [Features as a Glance](https://github.com/strands-agents/sdk-python?tab=readme-ov-file#features-at-a-glance) documentation.

In [ ]:
%%time


@tool
def word_count(text: str) -> int:
    """Count words in text.

    This docstring is used by the LLM to understand the tool's purpose.
    """
    return len(text.split())


bedrock_model = BedrockModel(
    boto_session=session,
    model_config={
        "model_id": "us.amazon.nova-micro-v1:0",
        "max_tokens": 256,
        "temperature": 0.2,
    },
)

agent = Agent(model=bedrock_model, tools=[word_count])
result = agent("How many words are in this sentence?")
print("\n")

## Web Research Agent Demonstration

Inspired by the AWS Blog Post, [Integrate dynamic web content in your generative AI application using a web search API and Amazon Bedrock Agents
](https://aws.amazon.com/blogs/machine-learning/integrate-dynamic-web-content-in-your-generative-ai-application-using-a-web-search-api-and-amazon-bedrock-agents/)

Requires a free [Serper](https://serper.dev/) and [Tavily](https://www.tavily.com/) API key. You must create a free account and obtain a API key for both serper.dev and tavily before continuing.

### Secure API Keys in AWS Secrets Manager

Secure your Serper and Tavily API keys in AWS Secrets Manager.

In [ ]:
# Initialize the Secrets Manager client
secrets_manager = session.client("secretsmanager")

try:
    # Create Serper API key secret
    serper_response = secrets_manager.create_secret(
        Name="SERPER_API_KEY",
        Description="The API secret key for Serper.",
        SecretString="<YOUR_SERPER_API_KEY>",
    )
    logger.info("Serper secret created:", serper_response["ARN"])

    # Create Tavily API key secret
    tavily_response = secrets_manager.create_secret(
        Name="TAVILY_API_KEY",
        Description="The API secret key for Tavily AI.",
        SecretString="<YOUR_TAVILY_API_KEY>",
    )
    logger.info("Tavily secret created:", tavily_response["ARN"])
except secrets_manager.exceptions.ClientError as e:
    if e.response["Error"]["Code"] == "ResourceExistsException":
        logger.warning("Secret already exists.")
    else:
        logger.error("An unexpected error occurred:", e)

### Retrieve API keys from AWS Secrets Manager

Retrieve your Serper and Tavily API keys from AWS Secrets Manager.

In [ ]:
def get_from_secretstore(key: str) -> str:
    try:
        secret_value = secrets_manager.get_secret_value(SecretId=key)
        logger.info(f"Retrieved {key} from Secrets Manager.")
        return secret_value["SecretString"]
    except secrets_manager.exceptions.ClientError as e:
        if e.response["Error"]["Code"] == "ResourceExistsException":
            logger.warning("Secret already exists.")
        else:
            logger.error(f"Could not get {key} from Secrets Manager: {e}")


# Retrieve API keys from Secrets Manager
SERPER_API_KEY = get_from_secretstore("SERPER_API_KEY")
TAVILY_API_KEY = get_from_secretstore("TAVILY_API_KEY")

### Define Custom Tools

Define Python functions as tools by using the `@tool` decorator.

In [ ]:
@tool
def google_search(search_query: str, target_website: str = "") -> str:
    """
    This tool performs a Google search using the Serper API.
    For targeted news, like 'what are the latest news in Austria' or similar.

    Args:
        search_query (str): The query to search for.
        target_website (str, optional): If provided, restricts the search to this website.
    Returns:
        str: The JSON response from the Serper API containing search results.
    """
    if SERPER_API_KEY is None:
        raise ValueError(
            "SERPER_API_KEY is not set. Please set the environment variable."
        )
    if not search_query:
        raise ValueError("Search query cannot be empty.")

    if target_website:
        search_query += f" site:{target_website}"

    conn = http.client.HTTPSConnection("google.serper.dev")
    payload = json.dumps({"q": search_query})
    headers = {"X-API-KEY": SERPER_API_KEY, "Content-Type": "application/json"}
    search_type = "news"  # "news", "search",

    try:
        conn.request("POST", f"/{search_type}", payload, headers)
        response = conn.getresponse()
        response_data = response.read().decode("utf-8")
        return response_data
    except http.client.HTTPException as e:
        logger.error(
            f"Failed to retrieve search results from Serper API, error: {e.code}"
        )

    return ""


@tool
def tavily_ai_search(search_query: str, target_website: str = "") -> str:
    """
    This tool performs a search using the Tavily AI Search API.
    To retrieve information via the Internet or for topics that the LLM does not know about and intense research is needed.

    Args:
        search_query (str): The query to search for.
        target_website (str, optional): If provided, restricts the search to this website.
    Returns:
        str: The JSON response from the Tavily AI Search API containing search results.
    """
    if TAVILY_API_KEY is None:
        raise ValueError(
            "TAVILY_API_KEY is not set. Please set the environment variable."
        )
    if not search_query:
        raise ValueError("Search query cannot be empty.")

    base_url = "https://api.tavily.com/search"
    headers = {"Content-Type": "application/json", "Accept": "application/json"}
    payload = {
        "api_key": TAVILY_API_KEY,
        "query": search_query,
        "search_depth": "advanced",
        "include_images": False,
        "include_answer": False,
        "include_raw_content": False,
        "max_results": 3,
        "include_domains": [target_website] if target_website else [],
        "exclude_domains": [],
    }

    data = json.dumps(payload).encode("utf-8")
    request = urllib.request.Request(base_url, data=data, headers=headers)

    try:
        response = urllib.request.urlopen(request)
        response_data = response.read().decode("utf-8")
        return response_data
    except urllib.error.HTTPError as e:
        logger.error(
            f"Failed to retrieve search results from Tavily AI Search, error: {e.code}"
        )

    return ""

### Test `tavily_ai_search` Tool Use

In [ ]:
%%time

bedrock_model = BedrockModel(
    boto_session=session,
    model_config={
        "model_id": "us.anthropic.claude-sonnet-4-20250514-v1:0",
        "max_tokens": 256,
        "temperature": 0.1,
    },
)

agent = Agent(model=bedrock_model, tools=[google_search, tavily_ai_search])
result = agent("What is the latest Anthropic Claude model?")
print("\n")

### Test `google_search` Tool Use

In [ ]:
%%time

bedrock_model = BedrockModel(
    boto_session=session,
    model_config={
        "model_id": "us.anthropic.claude-sonnet-4-20250514-v1:0",
        "max_tokens": 512,
        "temperature": 0.2,
    },
)

agent = Agent(model=bedrock_model, tools=[google_search, tavily_ai_search])
result = agent("What are the latest top 5 news headlines?")
print("\n")

### Multi-tool Use

In [ ]:
%%time

bedrock_model = BedrockModel(
    boto_session=session,
    model_config={
        "model_id": "us.amazon.nova-premier-v1:0",
        "max_tokens": 512,
        "temperature": 0.2,
    },
)

agent = Agent(model=bedrock_model, tools=[calculator, google_search, tavily_ai_search])

result = agent(
    "What is the square root of the current US National Debt divided by the current US population in USD? Show me the calculation steps and the final result."
)
print("\n")